In [1]:
%run -i ../header.py
from pylatex import Tabular, MultiColumn, NoEscape

# Export

In [2]:
def getPDGTrans(name):
    pdg_trans = pd.read_csv("../data/{}-trans-PDG.csv".format(name), header=0, index_col=[0,1])["frac_width"]\
                  .rename("PDG")
    pdg = pd.read_csv("../data/{}-spectrum.csv".format(name), header=0, index_col=0)["PDG_width"]
    for instate in pdg_trans.index.levels[0]:
        pdg_trans.loc[(instate, slice(None))] = pdg_trans[(instate, slice(None))].values*pdg[instate]*10**6
    pdg_trans.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in pdg_trans.index.values], name=["In", "Out"])
    return pdg_trans

In [3]:
def getDengTrans(cfgcode, cfgtag=""):
    suffix = "" if cfgtag == "" else "-"+cfgtag
    deng = pd.read_csv("../data/{}.deng2017{}.csv".format(cfgcode, suffix), header=0, index_col=[0,1])
    if sp.any(pd.isna(deng["Gtot"])):
        deng.loc[pd.isna(deng["Gtot"]), "Gtot"] = deng.loc[pd.isna(deng["Gtot"]).index.values, "Ge1"]
    if sp.any(pd.isna(deng["Gtot"])):
        deng.loc[pd.isna(deng["Gtot"]), "Gtot"] = deng.loc[pd.isna(deng["Gtot"]).index.values, "Gm1"]
    deng = deng["Gtot"].rename("Deng")
    deng *= 10**6
    deng.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in deng.index.values], name=["In", "Out"])
    return deng

In [4]:
def getRs(particle, prefix, cfgname, suffix=""):
    data = pd.read_csv("output/{}.{}/widths{}".format(prefix, cfgname, suffix), header=0, index_col = [0, 1])
    data *= 10**6
    data = data.loc[(data != 0).any(axis=1), :]
    data["in0"] /= data["width"]
    data["in2"] /= data["width"]
    data["out0"] /= data["width"]
    data["out2"] /= data["width"]
    data.index.names = ["In", "Out"]
    
    data_sub = data.query("In == '{0}'".format(particle))\
        .rename(columns={"out0": "$r^{(0)}$", "out2": "$r^{(2)}$"})\
        .drop(columns=["in0", "in2"])
    data_above = data.query("Out == '{0}'".format(particle))\
        .rename(columns={"in0": "$r^{(0)}$", "in2": "$r^{(2)}$"})\
        .drop(columns=["out0", "out2"])
    
    data = pd.concat([data_sub, data_above])
    data.columns = ["$\Gamma$", "$r^{(0)}$", "$r^{(2)}$"]
    
    data.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in data.index.values], name=["In", "Out"])
    return data

In [5]:
def reportRs(pname, prefix, cfgnames, suffixes=["", "E1"]): # allowed suffixes: "", "E1", "ELW"
    stack = []
    deng_stack = []
    for cfgname in cfgnames:
        cfgcode, cfgtag = cfgnameSplit(cfgname)
        parts = [getRs(pname, prefix, cfgname, suffix)\
                                    .add_prefix("" if suffix == "" else suffix+"-")\
                                for suffix in suffixes]
        parts = [part for part in parts if not part.empty]
        if len(parts) > 0:
            stack.append(pd.concat(parts, axis=1))
        deng_stack.append(getDengTrans(cfgcode, cfgtag))
    res = pd.concat(stack)
    res.insert(0, "Deng", pd.concat(deng_stack))
    res.insert(0, "PDG", getPDGTrans(cfgcode.split("-")[0]))
    res = dfsort(res, lambda a,b: cmpMultyStatesByCode(prefix, cfgcode
                                                      ,[name2code(i[1:-1]) for i in a]
                                                      ,[name2code(i[1:-1]) for i in b]))
    return res

## Widths

### $\psi(1S)$ and $\Upsilon(1S)$

In [6]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1S", "scr", ("c-scr",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_1S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.44,2.78,1,0,2.78,1,0
$\chi_{c0}(1P)$,$\psi(1S)$,133,179,144,1,0,179,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,285,319,331,0.571,0,378,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,371,292,501,0.142,0.519,434,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.29,2.91,1,0,2.91,1,0
$\chi_{c0}(2P)$,$\psi(1S)$,nan,2.3,1.64,1,0,2.68,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,88,88.8,0.629,0,112,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,247,0.17,0.47,194,0.1,0.6


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.44,2.78,1,0,nan,nan,nan
$\chi_{c0}(1P)$,$\psi(1S)$,133,179,179,1,0,187,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,285,319,378,0.5,0,411,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,371,292,434,0.1,0.6,481,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.29,2.91,1,0,nan,nan,nan
$\chi_{c0}(2P)$,$\psi(1S)$,nan,2.3,2.68,1,0,3.15,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,88,112,0.5,0,60.1,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,194,0.1,0.6,136,0.1,0.6


In [7]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1S", "lin", ("c-lin",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_1S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.39,2.57,1,0,2.57,1,0
$\chi_{c0}(1P)$,$\psi(1S)$,133,172,140,1,0,172,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,285,306,311,0.566,0,352,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,371,284,477,0.14,0.523,416,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.64,3.03,1,0,3.03,1,0
$\chi_{c0}(2P)$,$\psi(1S)$,nan,6.1,2.89,1,0,4.63,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,81,90.9,0.625,0,114,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,270,0.169,0.473,213,0.1,0.6


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.39,2.57,1,0,nan,nan,nan
$\chi_{c0}(1P)$,$\psi(1S)$,133,172,172,1,0,180,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,285,306,352,0.5,0,380,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,371,284,416,0.1,0.6,458,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.64,3.03,1,0,nan,nan,nan
$\chi_{c0}(2P)$,$\psi(1S)$,nan,6.1,4.63,1,0,1.71,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,81,114,0.5,0,58,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,213,0.1,0.6,146,0.1,0.6


In [8]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_1S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"), suffixes=suff)
    display(res)
    with open("export/width{}.yps_1S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,27.5,25,1,0,27.2,1,0
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,31.9,32.4,0.523,0,33.8,0.5,0
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,31.8,38.3,0.114,0.573,36.5,0.1,0.6
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,5.54,4.22,1,0,4.92,1,0
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,10.8,10.5,0.54,0,11.3,0.5,0
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,12.5,16.7,0.123,0.554,15.4,0.1,0.6
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,1.87,1.37,1,0,1.68,1,0
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,6.41,5.62,0.552,0,6.21,0.5,0
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,8.17,10.8,0.13,0.541,9.71,0.1,0.6


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,27.5,27.2,1,0,27.9,1,0
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,31.9,33.8,0.5,0,34.9,0.5,0
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,31.8,36.5,0.1,0.6,37.8,0.1,0.6
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,5.54,4.92,1,0,3.79,1,0
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,10.8,11.3,0.5,0,9.79,0.5,0
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,12.5,15.4,0.1,0.6,13.8,0.1,0.6
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,1.87,1.68,1,0,1.01,1,0
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,6.41,6.21,0.5,0,4.87,0.5,0
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,8.17,9.71,0.1,0.6,8.1,0.1,0.6


### $\psi(2S)$ and $\Upsilon(2S)$

In [9]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_2S", "scr", ("c-scr",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_2S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$   1.01   7.8      10.3          1          0   
                $\chi_{c0}(1P)$  29.6    22      33.1          1          0   
                $\chi_{c1}(1P)$  28.3    45      40.9      0.474          0   
                $\chi_{c2}(1P)$    27    46      40.6     0.0872      0.626   
                $\eta_{c}(2S)$  0.207  0.19     0.148          1          0   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    99      99.1          1          0   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   155       193      0.541          0   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   150       242      0.126       0.55   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   9.1      7.25          1          0   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    74      71.5      0.586          0   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    76       142      0.149      0.507   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi(2S)$      $\eta_{c}(1S)$          10.3             1             0  
                $\chi_{c0}(1P)$         28.1             1             0  
                $\chi_{c1}(1P)$         38.8           0.5             0  
                $\chi_{c2}(1P)$         42.4           0.1           0.6  
                $\eta_{c}(2S)$         0.148             1             0  
$\chi_{c0}(2P)$ $\psi(2S)$               112             1             0  
$\chi_{c1}(2P)$ $\psi(2S)$               208           0.5             0  
$\chi_{c2}(2P)$ $\psi(2S)$               222           0.1           0.6  
$\chi_{c0}(3P)$ $\psi(2S)$              9.98             1             0  
$\chi_{c1}(3P)$ $\psi(2S)$              83.8           0.5             0  
$\chi_{c2}(3P)$ $\psi(2S)$               120           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                      
$\psi(2S)$      $\eta_{c}(1S)$   1.01   7.8         10.3             1   
                $\chi_{c0}(1P)$  29.6    22         28.1             1   
                $\chi_{c1}(1P)$  28.3    45         38.8           0.5   
                $\chi_{c2}(1P)$    27    46         42.4           0.1   
                $\eta_{c}(2S)$  0.207  0.19        0.148             1   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    99          112             1   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   155          208           0.5   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   150          222           0.1   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   9.1         9.98             1   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    74         83.8           0.5   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    76          120           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi(2S)$      $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0          30.3              1   
                $\chi_{c1}(1P)$             0          39.9            0.5   
                $\chi_{c2}(1P)$           0.6          43.2            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi(2S)$                  0           117              1   
$\chi_{c1}(2P)$ $\psi(2S)$                  0           231            0.5   
$\chi_{c2}(2P)$ $\psi(2S)$                0.6           254            0.1   
$\chi_{c0}(3P)$ $\psi(2S)$                  0        0.0413              1   
$\chi_{c1}(3P)$ $\psi(2S)$                  0          41.7            0.5   
$\chi_{c2}(3P)$ $\psi(2S)$                0.6            78            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi(2S)$      $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi(2S)$                   0  
$\chi_{c1}(2P)$ $\psi(2S)$                   0  
$\chi_{c2}(2P)$ $\psi(2S)$                 0.6  
$\chi_{c0}(3P)$ $\psi(2S)$                   0  
$\chi_{c1}(3P)$ $\psi(2S)$                   0  
$\chi_{c2}(3P)$ $\psi(2S)$                 0.6

In [10]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_2S", "lin", ("c-lin",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_2S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$   1.01  8.08      9.63          1          0   
                $\chi_{c0}(1P)$  29.6    22      33.5          1          0   
                $\chi_{c1}(1P)$  28.3    42      41.9      0.474          0   
                $\chi_{c2}(1P)$    27    38      38.9     0.0877      0.625   
                $\eta_{c}(2S)$  0.207  0.19     0.158          1          0   
$\chi_{c0}(2P)$ $\psi(2S)$        nan   121       120          1          0   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   139       223      0.542          0   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   135       288      0.127      0.548   
$\chi_{c0}(3P)$ $\psi(2S)$        nan    17      11.3          1          0   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    94      90.7      0.596          0   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    97       201      0.155      0.497   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi(2S)$      $\eta_{c}(1S)$          9.63             1             0  
                $\chi_{c0}(1P)$         28.6             1             0  
                $\chi_{c1}(1P)$         39.8           0.5             0  
                $\chi_{c2}(1P)$         40.5           0.1           0.6  
                $\eta_{c}(2S)$         0.158             1             0  
$\chi_{c0}(2P)$ $\psi(2S)$               136             1             0  
$\chi_{c1}(2P)$ $\psi(2S)$               242           0.5             0  
$\chi_{c2}(2P)$ $\psi(2S)$               263           0.1           0.6  
$\chi_{c0}(3P)$ $\psi(2S)$              16.2             1             0  
$\chi_{c1}(3P)$ $\psi(2S)$               108           0.5             0  
$\chi_{c2}(3P)$ $\psi(2S)$               166           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                      
$\psi(2S)$      $\eta_{c}(1S)$   1.01  8.08         9.63             1   
                $\chi_{c0}(1P)$  29.6    22         28.6             1   
                $\chi_{c1}(1P)$  28.3    42         39.8           0.5   
                $\chi_{c2}(1P)$    27    38         40.5           0.1   
                $\eta_{c}(2S)$  0.207  0.19        0.158             1   
$\chi_{c0}(2P)$ $\psi(2S)$        nan   121          136             1   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   139          242           0.5   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   135          263           0.1   
$\chi_{c0}(3P)$ $\psi(2S)$        nan    17         16.2             1   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    94          108           0.5   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    97          166           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi(2S)$      $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0          30.7              1   
                $\chi_{c1}(1P)$             0          40.9            0.5   
                $\chi_{c2}(1P)$           0.6          41.2            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi(2S)$                  0           143              1   
$\chi_{c1}(2P)$ $\psi(2S)$                  0           269            0.5   
$\chi_{c2}(2P)$ $\psi(2S)$                0.6           303            0.1   
$\chi_{c0}(3P)$ $\psi(2S)$                  0         0.484              1   
$\chi_{c1}(3P)$ $\psi(2S)$                  0          38.8            0.5   
$\chi_{c2}(3P)$ $\psi(2S)$                0.6          89.6            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi(2S)$      $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi(2S)$                   0  
$\chi_{c1}(2P)$ $\psi(2S)$                   0  
$\chi_{c2}(2P)$ $\psi(2S)$                 0.6  
$\chi_{c0}(3P)$ $\psi(2S)$                   0  
$\chi_{c1}(3P)$ $\psi(2S)$                   0  
$\chi_{c2}(3P)$ $\psi(2S)$                 0.6

In [11]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_2S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"), suffixes=suff)
    display(res)
    with open("export/width{}.yps_2S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$ 1.22  1.09      0.93          1          0   
                $\chi_{b1}(1P)$ 2.21  2.17      1.44      0.494          0   
                $\chi_{b2}(1P)$ 2.29  2.62      1.57     0.0971      0.606   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan  14.4      12.5          1          0   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan  15.3      15.6      0.512          0   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan  15.3      17.4      0.108      0.585   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan  2.55      2.29          1          0   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan  5.63      5.43      0.526          0   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan  6.72      7.91      0.115       0.57   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\Upsilon(2S)$  $\chi_{b0}(1P)$        0.902             1             0  
                $\chi_{b1}(1P)$         1.43           0.5             0  
                $\chi_{b2}(1P)$         1.59           0.1           0.6  
$\chi_{b0}(2P)$ $\Upsilon(2S)$            13             1             0  
$\chi_{b1}(2P)$ $\Upsilon(2S)$            16           0.5             0  
$\chi_{b2}(2P)$ $\Upsilon(2S)$            17           0.1           0.6  
$\chi_{b0}(3P)$ $\Upsilon(2S)$          2.53             1             0  
$\chi_{b1}(3P)$ $\Upsilon(2S)$           5.7           0.5             0  
$\chi_{b2}(3P)$ $\Upsilon(2S)$          7.51           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                     
$\Upsilon(2S)$  $\chi_{b0}(1P)$ 1.22  1.09        0.902             1   
                $\chi_{b1}(1P)$ 2.21  2.17         1.43           0.5   
                $\chi_{b2}(1P)$ 2.29  2.62         1.59           0.1   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan  14.4           13             1   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan  15.3           16           0.5   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan  15.3           17           0.1   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan  2.55         2.53             1   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan  5.63          5.7           0.5   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan  6.72         7.51           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$             0         0.909              1   
                $\chi_{b1}(1P)$             0          1.43            0.5   
                $\chi_{b2}(1P)$           0.6          1.59            0.1   
$\chi_{b0}(2P)$ $\Upsilon(2S)$              0          13.3              1   
$\chi_{b1}(2P)$ $\Upsilon(2S)$              0          16.5            0.5   
$\chi_{b2}(2P)$ $\Upsilon(2S)$            0.6          17.7            0.1   
$\chi_{b0}(3P)$ $\Upsilon(2S)$              0          1.69              1   
$\chi_{b1}(3P)$ $\Upsilon(2S)$              0          4.52            0.5   
$\chi_{b2}(3P)$ $\Upsilon(2S)$            0.6          6.26            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\Upsilon(2S)$  $\chi_{b0}(1P)$              0  
                $\chi_{b1}(1P)$              0  
                $\chi_{b2}(1P)$            0.6  
$\chi_{b0}(2P)$ $\Upsilon(2S)$               0  
$\chi_{b1}(2P)$ $\Upsilon(2S)$               0  
$\chi_{b2}(2P)$ $\Upsilon(2S)$             0.6  
$\chi_{b0}(3P)$ $\Upsilon(2S)$               0  
$\chi_{b1}(3P)$ $\Upsilon(2S)$               0  
$\chi_{b2}(3P)$ $\Upsilon(2S)$             0.6

### $\psi(3S)$ and $\Upsilon(3S)$

In [12]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_3S", "scr", ("c-scr",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_3S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.76      9.62          1          0   
           $\chi_{c0}(1P)$  nan   6.7      19.2          1          0   
           $\chi_{c1}(1P)$  nan   6.7      9.78      0.427          0   
           $\chi_{c2}(1P)$  nan   2.5      1.93     0.0568      0.693   
           $\eta_{c}(2S)$   nan  1.78      2.01          1          0   
           $\chi_{c0}(2P)$  nan    27      32.1          1          0   
           $\chi_{c1}(2P)$  nan    67      45.7       0.48          0   
           $\chi_{c2}(2P)$  nan    82      45.2     0.0904      0.619   
           $\eta_{c}(3S)$   nan 0.088    0.0344          1          0   

                            E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In         Out                                                       
$\psi(3S)$ $\eta_{c}(1S)$          9.62             1             0  
           $\chi_{c0}(1P)$         13.7             1             0  
           $\chi_{c1}(1P)$         8.35           0.5             0  
           $\chi_{c2}(1P)$         2.23           0.1           0.6  
           $\eta_{c}(2S)$          2.01             1             0  
           $\chi_{c0}(2P)$         28.5             1             0  
           $\chi_{c1}(2P)$         43.9           0.5             0  
           $\chi_{c2}(2P)$         46.7           0.1           0.6  
           $\eta_{c}(3S)$        0.0344             1             0

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In         Out                                                     
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.76         9.62             1   
           $\chi_{c0}(1P)$  nan   6.7         13.7             1   
           $\chi_{c1}(1P)$  nan   6.7         8.35           0.5   
           $\chi_{c2}(1P)$  nan   2.5         2.23           0.1   
           $\eta_{c}(2S)$   nan  1.78         2.01             1   
           $\chi_{c0}(2P)$  nan    27         28.5             1   
           $\chi_{c1}(2P)$  nan    67         43.9           0.5   
           $\chi_{c2}(2P)$  nan    82         46.7           0.1   
           $\eta_{c}(3S)$   nan 0.088       0.0344             1   

                            E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$              0           nan            nan   
           $\chi_{c0}(1P)$             0          7.73              1   
           $\chi_{c1}(1P)$             0         0.675            0.5   
           $\chi_{c2}(1P)$           0.6          1.92            0.1   
           $\eta_{c}(2S)$              0           nan            nan   
           $\chi_{c0}(2P)$             0          31.4              1   
           $\chi_{c1}(2P)$             0          45.7            0.5   
           $\chi_{c2}(2P)$           0.6          47.9            0.1   
           $\eta_{c}(3S)$              0           nan            nan   

                            ELW-$r^{(2)}$  
In         Out                             
$\psi(3S)$ $\eta_{c}(1S)$             nan  
           $\chi_{c0}(1P)$              0  
           $\chi_{c1}(1P)$              0  
           $\chi_{c2}(1P)$            0.6  
           $\eta_{c}(2S)$             nan  
           $\chi_{c0}(2P)$              0  
           $\chi_{c1}(2P)$              0  
           $\chi_{c2}(2P)$            0.6  
           $\eta_{c}(3S)$             nan

In [13]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_3S", "lin", ("c-lin",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_3S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.66      10.4          1          0   
           $\chi_{c0}(1P)$  nan   5.9      21.6          1          0   
           $\chi_{c1}(1P)$  nan     4      10.1      0.425          0   
           $\chi_{c2}(1P)$  nan  0.25     0.775     0.0552      0.696   
           $\eta_{c}(2S)$   nan  1.65       2.8          1          0   
           $\chi_{c0}(2P)$  nan    19      42.9          1          0   
           $\chi_{c1}(2P)$  nan    55      65.3      0.478          0   
           $\chi_{c2}(2P)$  nan    67      63.4     0.0891      0.622   
           $\eta_{c}(3S)$   nan 0.051    0.0527          1          0   

                            E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In         Out                                                       
$\psi(3S)$ $\eta_{c}(1S)$          10.4             1             0  
           $\chi_{c0}(1P)$         15.3             1             0  
           $\chi_{c1}(1P)$         8.54           0.5             0  
           $\chi_{c2}(1P)$          0.9           0.1           0.6  
           $\eta_{c}(2S)$           2.8             1             0  
           $\chi_{c0}(2P)$         37.8             1             0  
           $\chi_{c1}(2P)$         62.4           0.5             0  
           $\chi_{c2}(2P)$         65.7           0.1           0.6  
           $\eta_{c}(3S)$        0.0527             1             0

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In         Out                                                     
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.66         10.4             1   
           $\chi_{c0}(1P)$  nan   5.9         15.3             1   
           $\chi_{c1}(1P)$  nan     4         8.54           0.5   
           $\chi_{c2}(1P)$  nan  0.25          0.9           0.1   
           $\eta_{c}(2S)$   nan  1.65          2.8             1   
           $\chi_{c0}(2P)$  nan    19         37.8             1   
           $\chi_{c1}(2P)$  nan    55         62.4           0.5   
           $\chi_{c2}(2P)$  nan    67         65.7           0.1   
           $\eta_{c}(3S)$   nan 0.051       0.0527             1   

                            E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$              0           nan            nan   
           $\chi_{c0}(1P)$             0           7.3              1   
           $\chi_{c1}(1P)$             0         0.105            0.5   
           $\chi_{c2}(1P)$           0.6           7.1            0.1   
           $\eta_{c}(2S)$              0           nan            nan   
           $\chi_{c0}(2P)$             0          41.9              1   
           $\chi_{c1}(2P)$             0          65.4            0.5   
           $\chi_{c2}(2P)$           0.6          67.7            0.1   
           $\eta_{c}(3S)$              0           nan            nan   

                            ELW-$r^{(2)}$  
In         Out                             
$\psi(3S)$ $\eta_{c}(1S)$             nan  
           $\chi_{c0}(1P)$              0  
           $\chi_{c1}(1P)$              0  
           $\chi_{c2}(1P)$            0.6  
           $\eta_{c}(2S)$             nan  
           $\chi_{c0}(2P)$              0  
           $\chi_{c1}(2P)$              0  
           $\chi_{c2}(2P)$            0.6  
           $\eta_{c}(3S)$             nan

In [14]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_3S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"), suffixes=suff)
    display(res)
    with open("export/width{}.yps_3S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG   Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                             
$\Upsilon(3S)$  $\chi_{b0}(1P)$ 0.0549  0.097    0.0999          1          0   
                $\chi_{b1}(1P)$ 0.0183 0.0005   8.3e-05      0.478          0   
                $\chi_{b2}(1P)$  0.201   0.14     0.149     0.0873      0.626   
                $\chi_{b0}(2P)$    1.2   1.21      1.32          1          0   
                $\chi_{b1}(2P)$   2.56   2.61      2.23      0.495          0   
                $\chi_{b2}(2P)$   2.66   3.16      2.43     0.0975      0.605   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan   7.95      9.47          1          0   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan   10.3      12.3      0.509          0   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan   10.8      13.6      0.106      0.589   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\Upsilon(3S)$  $\chi_{b0}(1P)$        0.091             1             0  
                $\chi_{b1}(1P)$     7.94e-05           0.5             0  
                $\chi_{b2}(1P)$        0.156           0.1           0.6  
                $\chi_{b0}(2P)$         1.29             1             0  
                $\chi_{b1}(2P)$         2.21           0.5             0  
                $\chi_{b2}(2P)$         2.45           0.1           0.6  
$\chi_{b0}(3P)$ $\Upsilon(3S)$          9.77             1             0  
$\chi_{b1}(3P)$ $\Upsilon(3S)$          12.5           0.5             0  
$\chi_{b2}(3P)$ $\Upsilon(3S)$          13.4           0.1           0.6

['E1', 'ELW']


PDG   Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                        
$\Upsilon(3S)$  $\chi_{b0}(1P)$ 0.0549  0.097        0.091             1   
                $\chi_{b1}(1P)$ 0.0183 0.0005     7.94e-05           0.5   
                $\chi_{b2}(1P)$  0.201   0.14        0.156           0.1   
                $\chi_{b0}(2P)$    1.2   1.21         1.29             1   
                $\chi_{b1}(2P)$   2.56   2.61         2.21           0.5   
                $\chi_{b2}(2P)$   2.66   3.16         2.45           0.1   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan   7.95         9.77             1   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan   10.3         12.5           0.5   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan   10.8         13.4           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\Upsilon(3S)$  $\chi_{b0}(1P)$             0        0.0394              1   
                $\chi_{b1}(1P)$             0        0.0357            0.5   
                $\chi_{b2}(1P)$           0.6         0.394            0.1   
                $\chi_{b0}(2P)$             0          1.31              1   
                $\chi_{b1}(2P)$             0          2.22            0.5   
                $\chi_{b2}(2P)$           0.6          2.46            0.1   
$\chi_{b0}(3P)$ $\Upsilon(3S)$              0            10              1   
$\chi_{b1}(3P)$ $\Upsilon(3S)$              0            13            0.5   
$\chi_{b2}(3P)$ $\Upsilon(3S)$            0.6            14            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\Upsilon(3S)$  $\chi_{b0}(1P)$              0  
                $\chi_{b1}(1P)$              0  
                $\chi_{b2}(1P)$            0.6  
                $\chi_{b0}(2P)$              0  
                $\chi_{b1}(2P)$              0  
                $\chi_{b2}(2P)$            0.6  
$\chi_{b0}(3P)$ $\Upsilon(3S)$               0  
$\chi_{b1}(3P)$ $\Upsilon(3S)$               0  
$\chi_{b2}(3P)$ $\Upsilon(3S)$             0.6

### $\psi_1(1D)$ which is $\psi(3770)$

In [15]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1_1D", "scr", ("c-scr",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_1_1D.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan     0.427          1          0   
                $\chi_{c0}(1P)$  190   261       275          1          0   
                $\chi_{c1}(1P)$ 67.5   135       119      0.539          0   
                $\chi_{c2}(1P)$  nan   8.1      5.74      0.253      0.531   
                $\eta_{c}(2S)$   nan   nan  0.000214          1          0   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    12      7.22          1          0   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   9.8      12.4      0.484          0   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.46     0.747       0.12      0.694   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.39     0.627          1          0   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     2      3.63      0.443          0   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.79      1.26      0.173     0.0518   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi_{1}(1D)$  $\eta_{c}(1S)$         0.427             1             0  
                $\chi_{c0}(1P)$          311             1             0  
                $\chi_{c1}(1P)$          143           0.5             0  
                $\chi_{c2}(1P)$         7.76           0.1           0.6  
                $\eta_{c}(2S)$      0.000214             1             0  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$          7.08             1             0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$          11.4           0.5             0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         0.607           0.1           0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$          0.56             1             0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$          2.84           0.5             0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$         0.218           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan        0.427             1   
                $\chi_{c0}(1P)$  190   261          311             1   
                $\chi_{c1}(1P)$ 67.5   135          143           0.5   
                $\chi_{c2}(1P)$  nan   8.1         7.76           0.1   
                $\eta_{c}(2S)$   nan   nan     0.000214             1   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    12         7.08             1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   9.8         11.4           0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.46        0.607           0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.39         0.56             1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     2         2.84           0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.79        0.218           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0           349              1   
                $\chi_{c1}(1P)$             0           153            0.5   
                $\chi_{c2}(1P)$           0.6          8.22            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$              0          7.12              1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$              0          11.8            0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$            0.6         0.638            0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$              0          8.48              1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$              0         0.355            0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$            0.6        0.0719            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi_{1}(1D)$  $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$             0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$             0.6

In [16]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1_1D", "lin", ("c-lin",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_1_1D.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan     0.365          1          0   
                $\chi_{c0}(1P)$  190   272       273          1          0   
                $\chi_{c1}(1P)$ 67.5   138       115      0.537          0   
                $\chi_{c2}(1P)$  nan   7.1      5.28      0.235      0.535   
                $\eta_{c}(2S)$   nan   nan  0.000138          1          0   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    20      17.7          1          0   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   7.9      18.3      0.482          0   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.36      1.09      0.131      0.712   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.27     0.126          1          0   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   3.2      6.71      0.434          0   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   1.5       2.7      0.201     0.0341   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi_{1}(1D)$  $\eta_{c}(1S)$         0.365             1             0  
                $\chi_{c0}(1P)$          307             1             0  
                $\chi_{c1}(1P)$          137           0.5             0  
                $\chi_{c2}(1P)$         7.03           0.1           0.6  
                $\eta_{c}(2S)$      0.000138             1             0  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$          17.2             1             0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$          16.7           0.5             0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         0.849           0.1           0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$          0.11             1             0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$          5.07           0.5             0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$         0.363           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan        0.365             1   
                $\chi_{c0}(1P)$  190   272          307             1   
                $\chi_{c1}(1P)$ 67.5   138          137           0.5   
                $\chi_{c2}(1P)$  nan   7.1         7.03           0.1   
                $\eta_{c}(2S)$   nan   nan     0.000138             1   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    20         17.2             1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   7.9         16.7           0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.36        0.849           0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.27         0.11             1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   3.2         5.07           0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   1.5        0.363           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0           339              1   
                $\chi_{c1}(1P)$             0           146            0.5   
                $\chi_{c2}(1P)$           0.6          7.39            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$              0          17.4              1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$              0          17.4            0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$            0.6         0.908            0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$              0          8.75              1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$              0         0.328            0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$            0.6        0.0873            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi_{1}(1D)$  $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$             0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$             0.6

## Sumrules

In [17]:
def getMasses(prefix, cfgname):
    with open("../quarkEigen/output/{}.{}/config".format(prefix, cfgname), "r") as f:
        mass_cfg = json.load(f, object_pairs_hook=OrderedDict)
    with open("../quarkEigen/output/{}.{}/exclude".format(prefix, cfgname), "r") as f:
        mass_excl = f.read().split("\n")
    mass_data = OrderedDict()
    for particle, pdata in mass_cfg.items():
        if particle not in mass_excl:
            continue
        energ_data = pd.read_csv("../quarkEigen/output/{}.{}/data/{}/minE.dat".format(prefix, cfgname, particle))\
                     .iloc[-1, 1]
        energ = pdata["eq"]["env"]["mC"]*2 + energ_data
        mass_data.update({particle: energ})
    cfgcode, cfgtag = cfgnameSplit(cfgname)
    mass_data = pd.DataFrame.from_dict(mass_data, orient="index")
    mass_data.columns = ["E"]
    pdg = pd.read_csv("../data/{}-spectrum.csv".format(cfgcode.split("-")[0]), header=0, index_col=0)["PDG"]/10**3
    pdg = pdg.loc[pdg.index.intersection(mass_data.index).values]
    mass_data.loc[~pd.isna(pdg), "E"] = pdg.loc[~pd.isna(pdg)]
    mass_data.index = pd.Index(["${}$".format(code2name(k)) for k in mass_data.index.values], name="States")
    return mass_data

In [18]:
def getPprops(prefix, cfgname):
    with open("../quarkEigen/output/{}.{}/config".format(prefix, cfgname), "r") as f:
        pprops = json.load(f, object_pairs_hook=OrderedDict)
    props = pd.DataFrame(OrderedDict([(particle, values["eq"]) for particle, values in pprops.items()])).T
    props.drop(columns=["env", "E"], inplace=True)
    props.index = pd.Index([r"${}$".format(code2name(particle)) for particle in pprops.keys()], name="States")
    return props

In [19]:
def getSumrule(pname, prefix, cfgnames, suffix=""):
    cfgcode, cfgtag = cfgnameSplit(cfgnames[0])
    msuffix = "" if suffix == "" else suffix+"-"
    pretty_pname = "${}$".format(code2name(pname))
    
    widths = reportRs(pname, prefix, cfgnames, [suffix])
    masses = getMasses(prefix, cfgcode)
    pprops = getPprops(prefix, cfgcode)
    
    sumrule = []

    abovethr = [p[0] for p in widths.query("Out == r'{}'".format(pretty_pname)).index.values]
    sumrule.append(
        8*sp.pi**2*masses.loc[abovethr, "E"]**3\
        /(masses.loc[abovethr, "E"]**2 - masses.loc[pretty_pname, "E"]**2)**3\
        *pprops.loc[abovethr, "xJ"]
    )
    sumrule[0].index = pd.Index([(v, pretty_pname) for v in sumrule[0].index.values])
    
    belowthr = [p[1] for p in widths.query("In == r'{}'".format(pretty_pname)).index.values]
    sumrule.append(
        8*sp.pi**2*masses.loc[pretty_pname, "E"]**3\
        /(masses.loc[pretty_pname, "E"]**2 - masses.loc[belowthr, "E"]**2)**3\
        *pprops.loc[pretty_pname, "xJ"]
    )
    sumrule[1].index = pd.Index([(pretty_pname, v) for v in sumrule[1].index.values])
    sr = pd.concat(sumrule)*(0.19732697)**2*10**4*10**(-6)
    sr = sr.reindex(widths.index.values)
    
    widths["$SR-PDG$"] = widths["PDG"]
    if sp.any(pd.isna(widths["PDG"])):
        widths.loc[pd.isna(widths["PDG"]), "$SR-PDG$"] = widths.loc[pd.isna(widths["PDG"]).index.values, msuffix+"$\Gamma$"]
    widths.loc[:, "$SR-PDG$"] *= sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths["$SR-Deng$"] = widths["Deng"]
    if sp.any(pd.isna(widths["Deng"])):
        widths.loc[pd.isna(widths["Deng"]), "$SR-Deng$"] = widths.loc[pd.isna(widths["Deng"]).index.values, msuffix+"$\Gamma$"]
    widths.loc[:, "$SR-Deng$"] *= sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths["$SR-\Gamma$"] = widths[msuffix+"$\Gamma$"]*sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths.drop(columns=[msuffix+'$r^{(2)}$', msuffix+'$r^{(0)}$'], inplace=True)
    widths = widths[[widths.columns[i] for i in [0, 3, 1, 4, 2, 5]]]
    widths = dfsort(widths, lambda a,b: cmpMultyStatesByCode(prefix, cfgcode
                                                         ,[name2code(i[1:-1]) for i in a]
                                                         ,[name2code(i[1:-1]) for i in b]))
    
    sigma = (widths["$SR-Deng$"] - widths["$SR-\Gamma$"]).std()
    
    return widths, pd.DataFrame(widths[["$SR-PDG$", "$SR-Deng$", "$SR-\Gamma$"]].sum().rename("$SR$")), sigma

In [24]:
def reportSumrules(pname, prefix, cfgnames, suffixes=("", "E1", "ELW"), sigma=0):
    cfgcode, cfgtag = cfgnameSplit(cfgnames[0])
    if sigma is not None:
        try:
            sigma = list(sigma)
        except TypeError:
            sigma = [sigma]*len(suffixes)
        sigma = dict(zip(suffixes, sigma))
    for approx in suffixes:
        display("Approximation: "+(approx if len(approx) > 0 else "tot"))
        report, sr, autosigma = getSumrule(pname, prefix, cfgnames, approx)
        display(sr)
        sigma[approx] = autosigma if sigma[approx] == 0 else sigma[approx]
        display(sigma[approx])
        display(report)
        
        tab = Tabular("|l|l|c|c|c|c|c|c|")
        
        # Header
        tab.add_hline()
        tab.add_row(["", ""] + [NoEscape(i) for i in report.columns])
        tab.add_row([NoEscape(i) for i in report.index.names] + [""]*6)
        tab.add_hline()
        
        curN = "1"
        tot = report.iloc[0, 1::2].fillna(0)*0
        getN = re.compile("\((\d+)[A-Z]\)\$$")
        for entry in report.index.values:
            # Manage subtotals
            label = entry[0] if entry[0] != "${}$".format(code2name(pname)) else entry[1]
            newN = getN.search(label).groups()[0]
            if newN != curN:
                tab.add_hline()
                tab.add_hline()
                try:
                    tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                            [MultiColumn(2, align="|r|", data=NoEscape("${:.2f} ({:.2f}\sigma)$".format(num, num/sigma[approx]))) for num in tot])
                except:
                    tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                            [MultiColumn(2, align="|r|", data="{:.2f}".format(num)) for num in tot])                
                tab.add_hline()
                tab.add_hline()
                tot = report.loc[entry].fillna(0).values[1::2]
                curN = newN
            else:
                tot += report.loc[entry].fillna(0).values[1::2]
            
            # Regular entry
            tab.add_row([NoEscape(i) for i in list(entry) + ["{:.2f}".format(num) for num in report.loc[entry]]])
            
        # Final subtotal
        tab.add_hline()
        tab.add_hline()
        try:
            tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                    [MultiColumn(2, align="|r|", data=NoEscape("${:.2f}~({:.2f}\sigma)$".format(num, num/sigma[approx]))) for num in tot])
        except:
            tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                    [MultiColumn(2, align="|r|", data="{:.2f}".format(num)) for num in tot])                
        tab.add_hline()
        tab.add_hline()
        
        # Total
        try:
            tab.add_row([MultiColumn(2, align="|c|", data="Total")] +\
                    [MultiColumn(2, align="|r|", data=NoEscape("${:.2f}~({:.2f}\sigma)$".format(num, num/sigma[approx]))) for num in sr.iloc[:, 0]])
        except:
            tab.add_row([MultiColumn(2, align="|c|", data="Total")] +\
                    [MultiColumn(2, align="|r|", data="{:.2f}".format(num)) for num in sr.iloc[:, 0]])                
        tab.add_hline()
        
        # with open("export/sr{}.{}.{}.tex".format(approx, pname, cfgcode), "w") as f:
        #     f.write(sr.to_latex(escape=False, bold_rows=True, column_format="l|c"))
        # with open("export/sr{}-report.{}.{}.tex".format(approx, pname, cfgcode), "w") as f:
        #     f.write(report.to_latex(escape=False, bold_rows=True, column_format="l|l|c|c|c|c|c|c"))
        tab.generate_tex("export/sr{}-report.{}.{}".format(approx, pname, cfgcode))

## Bottomonium sumrules

### $\Upsilon(1S)$

In [25]:
reportSumrules("yps_1S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"), sigma=[0]*3)

'Approximation: tot'

,$SR$
$SR-PDG$,-0.113
$SR-Deng$,-1.02
$SR-\Gamma$,-0.113


0.2181217797830497

,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.61,27.5,-1.77,25,-1.61
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.57,31.9,-2.53,32.4,-2.57
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,3.92,31.8,3.25,38.3,3.92
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.0394,5.54,-0.0517,4.22,-0.0394
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.146,10.8,-0.151,10.5,-0.146
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.295,12.5,0.221,16.7,0.295
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.00558,1.87,-0.00762,1.37,-0.00558
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0359,6.41,-0.0409,5.62,-0.0359
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,0.0815,8.17,0.0619,10.8,0.0815


'Approximation: E1'

...

,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.75,27.5,-1.77,27.2,-1.75
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.57,31.9,-2.42,33.8,-2.57
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,4.07,31.8,3.55,36.5,4.07
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.046,5.54,-0.0517,4.92,-0.046
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.146,10.8,-0.14,11.3,-0.146
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.316,12.5,0.256,15.4,0.316
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.00686,1.87,-0.00762,1.68,-0.00686
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0359,6.41,-0.037,6.21,-0.0359
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,0.0895,8.17,0.0753,9.71,0.0895


'Approximation: ELW'

,$SR$
$SR-PDG$,-0.0587
$SR-Deng$,-0.549
$SR-\Gamma$,-0.0587


0.2445791636339477

,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.79,27.5,-1.77,27.9,-1.79
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.65,31.9,-2.42,34.9,-2.65
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,4.22,31.8,3.55,37.8,4.22
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.0354,5.54,-0.0517,3.79,-0.0354
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.127,10.8,-0.14,9.79,-0.127
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.283,12.5,0.256,13.8,0.283
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.00412,1.87,-0.00762,1.01,-0.00412
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0281,6.41,-0.037,4.87,-0.0281
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,0.0746,8.17,0.0753,8.1,0.0746


### $\Upsilon(2S)$

In [26]:
reportSumrules("yps_2S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"), sigma=[0]*3)

'Approximation: tot'

,$SR$
$SR-PDG$,-0.791
$SR-Deng$,-1.04
$SR-\Gamma$,-1.12


1.847018282376902

PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                       
$\Upsilon(2S)$  $\chi_{b0}(1P)$ 1.22    -3.26  1.09     -2.92      0.93   
                $\chi_{b1}(1P)$ 2.21    -5.84  2.17     -5.74      1.44   
                $\chi_{b2}(1P)$ 2.29     9.97  2.62      11.4      1.57   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan    -5.33  14.4     -6.16      12.5   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan    -7.63  15.3     -7.49      15.6   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan     11.1  15.3      9.77      17.4   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan  -0.0916  2.55    -0.102      2.29   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan   -0.302  5.63    -0.313      5.43   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan    0.575  6.72     0.489      7.91   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$       -2.49  
                $\chi_{b1}(1P)$       -3.82  
                $\chi_{b2}(1P)$        6.86  
$\chi_{b0}(2P)$ $\Upsilon(2S)$        -5.33  
$\chi_{b1}(2P)$ $\Upsilon(2S)$        -7.63  
$\chi_{b2}(2P)$ $\Upsilon(2S)$         11.1  
$\chi_{b0}(3P)$ $\Upsilon(2S)$      -0.0916  
$\chi_{b1}(3P)$ $\Upsilon(2S)$       -0.302  
$\chi_{b2}(3P)$ $\Upsilon(2S)$        0.575

'Approximation: E1'

...

PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$ 1.22    -3.26  1.09     -2.92        0.902   
                $\chi_{b1}(1P)$ 2.21     -5.9  2.17     -5.81         1.43   
                $\chi_{b2}(1P)$ 2.29      9.8  2.62      11.2         1.59   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan    -5.57  14.4     -6.16           13   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan    -7.63  15.3     -7.31           16   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan     11.4  15.3      10.2           17   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan   -0.101  2.55    -0.102         2.53   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan   -0.302  5.63    -0.298          5.7   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan    0.601  6.72     0.538         7.51   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$       -2.42  
                $\chi_{b1}(1P)$       -3.82  
                $\chi_{b2}(1P)$         6.8  
$\chi_{b0}(2P)$ $\Upsilon(2S)$        -5.57  
$\chi_{b1}(2P)$ $\Upsilon(2S)$        -7.63  
$\chi_{b2}(2P)$ $\Upsilon(2S)$         11.4  
$\chi_{b0}(3P)$ $\Upsilon(2S)$       -0.101  
$\chi_{b1}(3P)$ $\Upsilon(2S)$       -0.302  
$\chi_{b2}(3P)$ $\Upsilon(2S)$        0.601

'Approximation: ELW'

,$SR$
$SR-PDG$,-0.944
$SR-Deng$,-0.593
$SR-\Gamma$,-1.03


1.8220486745506932

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\Upsilon(2S)$  $\chi_{b0}(1P)$ 1.22    -3.26  1.09     -2.92         0.909   
                $\chi_{b1}(1P)$ 2.21     -5.9  2.17     -5.81          1.43   
                $\chi_{b2}(1P)$ 2.29      9.8  2.62      11.2          1.59   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan     -5.7  14.4     -6.16          13.3   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan     -7.9  15.3     -7.31          16.5   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan     11.8  15.3      10.2          17.7   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan  -0.0675  2.55    -0.102          1.69   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan   -0.239  5.63    -0.298          4.52   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan    0.502  6.72     0.538          6.26   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$       -2.44  
                $\chi_{b1}(1P)$       -3.83  
                $\chi_{b2}(1P)$        6.83  
$\chi_{b0}(2P)$ $\Upsilon(2S)$         -5.7  
$\chi_{b1}(2P)$ $\Upsilon(2S)$         -7.9  
$\chi_{b2}(2P)$ $\Upsilon(2S)$         11.8  
$\chi_{b0}(3P)$ $\Upsilon(2S)$      -0.0675  
$\chi_{b1}(3P)$ $\Upsilon(2S)$       -0.239  
$\chi_{b2}(3P)$ $\Upsilon(2S)$        0.502

### $\Upsilon(3S)$

In [27]:
reportSumrules("yps_3S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"))

'Approximation: tot'

,$SR$
$SR-PDG$,-6.52
$SR-Deng$,-2
$SR-\Gamma$,-7.61


3.3546899774029

PDG $SR-PDG$   Deng $SR-Deng$  $\Gamma$  \
In              Out                                                          
$\Upsilon(3S)$  $\chi_{b0}(1P)$ 0.0549 -0.00557  0.097  -0.00986    0.0999   
                $\chi_{b1}(1P)$ 0.0183 -0.00109 0.0005 -2.99e-05   8.3e-05   
                $\chi_{b2}(1P)$  0.201   0.0153   0.14    0.0107     0.149   
                $\chi_{b0}(2P)$    1.2    -7.75   1.21     -7.82      1.32   
                $\chi_{b1}(2P)$   2.56    -14.8   2.61     -15.1      2.23   
                $\chi_{b2}(2P)$   2.66     24.8   3.16      29.4      2.43   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan    -14.6   7.95     -12.3      9.47   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan      -19   10.3       -16      12.3   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan     24.9   10.8      19.8      13.6   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(3S)$  $\chi_{b0}(1P)$     -0.0102  
                $\chi_{b1}(1P)$   -4.97e-06  
                $\chi_{b2}(1P)$      0.0114  
                $\chi_{b0}(2P)$       -8.56  
                $\chi_{b1}(2P)$       -12.9  
                $\chi_{b2}(2P)$        22.6  
$\chi_{b0}(3P)$ $\Upsilon(3S)$        -14.6  
$\chi_{b1}(3P)$ $\Upsilon(3S)$          -19  
$\chi_{b2}(3P)$ $\Upsilon(3S)$         24.9

'Approximation: E1'

...

PDG $SR-PDG$   Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                             
$\Upsilon(3S)$  $\chi_{b0}(1P)$ 0.0549 -0.00557  0.097  -0.00986        0.091   
                $\chi_{b1}(1P)$ 0.0183 -0.00114 0.0005 -3.13e-05     7.94e-05   
                $\chi_{b2}(1P)$  0.201   0.0142   0.14    0.0099        0.156   
                $\chi_{b0}(2P)$    1.2    -7.75   1.21     -7.82         1.29   
                $\chi_{b1}(2P)$   2.56      -15   2.61     -15.3         2.21   
                $\chi_{b2}(2P)$   2.66     24.4   3.16        29         2.45   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan    -15.1   7.95     -12.3         9.77   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan      -19   10.3     -15.7         12.5   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan     25.3   10.8      20.5         13.4   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(3S)$  $\chi_{b0}(1P)$    -0.00924  
                $\chi_{b1}(1P)$   -4.97e-06  
                $\chi_{b2}(1P)$       0.011  
                $\chi_{b0}(2P)$       -8.34  
                $\chi_{b1}(2P)$       -12.9  
                $\chi_{b2}(2P)$        22.4  
$\chi_{b0}(3P)$ $\Upsilon(3S)$        -15.1  
$\chi_{b1}(3P)$ $\Upsilon(3S)$          -19  
$\chi_{b2}(3P)$ $\Upsilon(3S)$         25.3

'Approximation: ELW'

,$SR$
$SR-PDG$,-7.03
$SR-Deng$,-1.6
$SR-\Gamma$,-7.61


3.6383254364129307

PDG $SR-PDG$   Deng $SR-Deng$  \
In              Out                                                
$\Upsilon(3S)$  $\chi_{b0}(1P)$ 0.0549 -0.00557  0.097  -0.00986   
                $\chi_{b1}(1P)$ 0.0183 -0.00114 0.0005 -3.13e-05   
                $\chi_{b2}(1P)$  0.201   0.0142   0.14    0.0099   
                $\chi_{b0}(2P)$    1.2    -7.75   1.21     -7.82   
                $\chi_{b1}(2P)$   2.56      -15   2.61     -15.3   
                $\chi_{b2}(2P)$   2.66     24.4   3.16        29   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan    -15.5   7.95     -12.3   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan    -19.7   10.3     -15.7   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan     26.5   10.8      20.5   

                                 ELW-$\Gamma$ $SR-\Gamma$  
In              Out                                        
$\Upsilon(3S)$  $\chi_{b0}(1P)$        0.0394      -0.004  
                $\chi_{b1}(1P)$        0.0357    -0.00223  
                $\chi_{b2}(1P)$         0.394      0.0279  
                $\chi_{b0}(2P)$          1.31       -8.46  
                $\chi_{b1}(2P)$          2.22         -13  
                $\chi_{b2}(2P)$          2.46        22.6  
$\chi_{b0}(3P)$ $\Upsilon(3S)$             10       -15.5  
$\chi_{b1}(3P)$ $\Upsilon(3S)$             13       -19.7  
$\chi_{b2}(3P)$ $\Upsilon(3S)$             14        26.5

## Charmonium sumrules

### $\psi(1S)$

In [28]:
reportSumrules("psi_1S", "scr", ("c-scr",), sigma=6.7)

'Approximation: tot'

,$SR$
$SR-PDG$,-27.8
$SR-Deng$,-54.5
$SR-\Gamma$,-32.7


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.44,-20.6,2.78,-23.5
$\chi_{c0}(1P)$,$\psi(1S)$,133,-18.4,179,-24.7,144,-19.9
$\chi_{c1}(1P)$,$\psi(1S)$,285,-31.7,319,-35.5,331,-36.9
$\chi_{c2}(1P)$,$\psi(1S)$,371,33.9,292,26.7,501,45.9
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0887,2.29,-0.0697,2.91,-0.0887
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0159,2.3,-0.0223,1.64,-0.0159
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.64,88,-1.63,88.8,-1.64
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.48,93,1.31,247,3.48


'Approximation: E1'

...

,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.44,-20.6,2.78,-23.5
$\chi_{c0}(1P)$,$\psi(1S)$,133,-18.4,179,-24.7,179,-24.7
$\chi_{c1}(1P)$,$\psi(1S)$,285,-27.8,319,-31.1,378,-36.9
$\chi_{c2}(1P)$,$\psi(1S)$,371,45,292,35.4,434,52.7
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0887,2.29,-0.0697,2.91,-0.0887
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0259,2.3,-0.0223,2.68,-0.0259
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.64,88,-1.3,112,-1.64
$\chi_{c2}(2P)$,$\psi(1S)$,nan,4.55,93,2.18,194,4.55


'Approximation: ELW'

,$SR$
$SR-PDG$,1.11
$SR-Deng$,-19.5
$SR-\Gamma$,-5.22


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{c0}(1P)$,$\psi(1S)$,133,-18.4,179,-24.7,187,-25.8
$\chi_{c1}(1P)$,$\psi(1S)$,285,-27.8,319,-31.1,411,-40.1
$\chi_{c2}(1P)$,$\psi(1S)$,371,45,292,35.4,481,58.4
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0305,2.3,-0.0223,3.15,-0.0305
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-0.884,88,-1.3,60.1,-0.884
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.2,93,2.18,136,3.2


In [29]:
reportSumrules("psi_1S", "lin", ("c-lin",), sigma=6.7)

'Approximation: tot'

,$SR$
$SR-PDG$,-26.5
$SR-Deng$,-51.5
$SR-\Gamma$,-28.9


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.39,-20.2,2.57,-21.7
$\chi_{c0}(1P)$,$\psi(1S)$,133,-18.4,172,-23.7,140,-19.3
$\chi_{c1}(1P)$,$\psi(1S)$,285,-31.4,306,-33.8,311,-34.3
$\chi_{c2}(1P)$,$\psi(1S)$,371,34.4,284,26.4,477,44.3
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0923,2.64,-0.0804,3.03,-0.0923
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0279,6.1,-0.059,2.89,-0.0279
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.55,81,-1.38,90.9,-1.55
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.86,93,1.33,270,3.86


'Approximation: E1'

...

,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.39,-20.2,2.57,-21.7
$\chi_{c0}(1P)$,$\psi(1S)$,133,-18.4,172,-23.7,172,-23.8
$\chi_{c1}(1P)$,$\psi(1S)$,285,-27.8,306,-29.8,352,-34.3
$\chi_{c2}(1P)$,$\psi(1S)$,371,45,284,34.5,416,50.4
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0923,2.64,-0.0804,3.03,-0.0923
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0449,6.1,-0.059,4.63,-0.0449
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.55,81,-1.1,114,-1.55
$\chi_{c2}(2P)$,$\psi(1S)$,nan,5,93,2.18,213,5


'Approximation: ELW'

,$SR$
$SR-PDG$,1.45
$SR-Deng$,-18.1
$SR-\Gamma$,-3.58


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{c0}(1P)$,$\psi(1S)$,133,-18.4,172,-23.7,180,-24.8
$\chi_{c1}(1P)$,$\psi(1S)$,285,-27.8,306,-29.8,380,-37
$\chi_{c2}(1P)$,$\psi(1S)$,371,45,284,34.5,458,55.6
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0166,6.1,-0.059,1.71,-0.0166
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-0.79,81,-1.1,58,-0.79
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.43,93,2.18,146,3.43


### $\psi(2S)$

In [30]:
reportSumrules("psi_2S", "scr", ("c-scr",), sigma=16.6)

'Approximation: tot'

,$SR$
$SR-PDG$,20.9
$SR-Deng$,24.5
$SR-\Gamma$,51.7


16.6

PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi(2S)$      $\eta_{c}(1S)$   1.01  -0.0453   7.8    -0.351      10.3   
                $\chi_{c0}(1P)$  29.6    -19.2    22     -14.3      33.1   
                $\chi_{c1}(1P)$  28.3      -31    45     -49.3      40.9   
                $\chi_{c2}(1P)$    27     80.5    46       137      40.6   
                $\eta_{c}(2S)$  0.207    -23.5  0.19     -21.5     0.148   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -33.4    99     -33.3      99.1   
$\chi_{c1}(2P)$ $\psi(2S)$        nan     -111   155     -89.2       193   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      155   150      95.9       242   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.339   9.1    -0.425      7.25   
$\chi_{c1}(3P)$ $\psi(2S)$        nan     -4.5    74     -4.65      71.5   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     8.19    76      4.39       142   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.462  
                $\chi_{c0}(1P)$       -21.5  
                $\chi_{c1}(1P)$       -44.8  
                $\chi_{c2}(1P)$         121  
                $\eta_{c}(2S)$        -16.8  
$\chi_{c0}(2P)$ $\psi(2S)$            -33.4  
$\chi_{c1}(2P)$ $\psi(2S)$             -111  
$\chi_{c2}(2P)$ $\psi(2S)$              155  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.339  
$\chi_{c1}(3P)$ $\psi(2S)$             -4.5  
$\chi_{c2}(3P)$ $\psi(2S)$             8.19

'Approximation: E1'

...

PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$   1.01  -0.0453   7.8    -0.351         10.3   
                $\chi_{c0}(1P)$  29.6    -19.2    22     -14.3         28.1   
                $\chi_{c1}(1P)$  28.3    -32.7    45       -52         38.8   
                $\chi_{c2}(1P)$    27     74.6    46       127         42.4   
                $\eta_{c}(2S)$  0.207    -23.5  0.19     -21.5        0.148   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -37.6    99     -33.3          112   
$\chi_{c1}(2P)$ $\psi(2S)$        nan     -111   155     -82.5          208   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      168   150       113          222   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.466   9.1    -0.425         9.98   
$\chi_{c1}(3P)$ $\psi(2S)$        nan     -4.5    74     -3.97         83.8   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     9.68    76      6.14          120   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.462  
                $\chi_{c0}(1P)$       -18.2  
                $\chi_{c1}(1P)$       -44.8  
                $\chi_{c2}(1P)$         117  
                $\eta_{c}(2S)$        -16.8  
$\chi_{c0}(2P)$ $\psi(2S)$            -37.6  
$\chi_{c1}(2P)$ $\psi(2S)$             -111  
$\chi_{c2}(2P)$ $\psi(2S)$              168  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.466  
$\chi_{c1}(3P)$ $\psi(2S)$             -4.5  
$\chi_{c2}(3P)$ $\psi(2S)$             9.68

'Approximation: ELW'

,$SR$
$SR-PDG$,55.8
$SR-Deng$,60
$SR-\Gamma$,86.8


16.6

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\chi_{c0}(1P)$ 29.6    -19.2    22     -14.3          30.3   
                $\chi_{c1}(1P)$ 28.3    -32.7    45       -52          39.9   
                $\chi_{c2}(1P)$   27     74.6    46       127          43.2   
$\chi_{c0}(2P)$ $\psi(2S)$       nan    -39.4    99     -33.3           117   
$\chi_{c1}(2P)$ $\psi(2S)$       nan     -123   155     -82.5           231   
$\chi_{c2}(2P)$ $\psi(2S)$       nan      191   150       113           254   
$\chi_{c0}(3P)$ $\psi(2S)$       nan -0.00193   9.1    -0.425        0.0413   
$\chi_{c1}(3P)$ $\psi(2S)$       nan    -2.23    74     -3.97          41.7   
$\chi_{c2}(3P)$ $\psi(2S)$       nan      6.3    76      6.14            78   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\chi_{c0}(1P)$       -19.6  
                $\chi_{c1}(1P)$       -46.1  
                $\chi_{c2}(1P)$         120  
$\chi_{c0}(2P)$ $\psi(2S)$            -39.4  
$\chi_{c1}(2P)$ $\psi(2S)$             -123  
$\chi_{c2}(2P)$ $\psi(2S)$              191  
$\chi_{c0}(3P)$ $\psi(2S)$         -0.00193  
$\chi_{c1}(3P)$ $\psi(2S)$            -2.23  
$\chi_{c2}(3P)$ $\psi(2S)$              6.3

In [31]:
reportSumrules("psi_2S", "lin", ("c-lin",), sigma=16.6)

'Approximation: tot'

,$SR$
$SR-PDG$,54.8
$SR-Deng$,14.3
$SR-\Gamma$,77.8


16.6

PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi(2S)$      $\eta_{c}(1S)$   1.01  -0.0453  8.08    -0.364      9.63   
                $\chi_{c0}(1P)$  29.6    -19.2    22     -14.3      33.5   
                $\chi_{c1}(1P)$  28.3      -31    42     -46.1      41.9   
                $\chi_{c2}(1P)$    27     80.2    38       113      38.9   
                $\eta_{c}(2S)$  0.207    -23.5  0.19     -21.5     0.158   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -40.3   121     -40.8       120   
$\chi_{c1}(2P)$ $\psi(2S)$        nan    -97.1   139     -60.4       223   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      183   135      85.5       288   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.329    17    -0.493      11.3   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -3.62    94     -3.75      90.7   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     6.82    97       3.3       201   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.433  
                $\chi_{c0}(1P)$       -21.7  
                $\chi_{c1}(1P)$         -46  
                $\chi_{c2}(1P)$         116  
                $\eta_{c}(2S)$        -17.9  
$\chi_{c0}(2P)$ $\psi(2S)$            -40.3  
$\chi_{c1}(2P)$ $\psi(2S)$            -97.1  
$\chi_{c2}(2P)$ $\psi(2S)$              183  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.329  
$\chi_{c1}(3P)$ $\psi(2S)$            -3.62  
$\chi_{c2}(3P)$ $\psi(2S)$             6.82

'Approximation: E1'

...

PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$   1.01  -0.0453  8.08    -0.364         9.63   
                $\chi_{c0}(1P)$  29.6    -19.2    22     -14.3         28.6   
                $\chi_{c1}(1P)$  28.3    -32.7    42     -48.6         39.8   
                $\chi_{c2}(1P)$    27     74.6    38       105         40.5   
                $\eta_{c}(2S)$  0.207    -23.5  0.19     -21.5        0.158   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -45.8   121     -40.8          136   
$\chi_{c1}(2P)$ $\psi(2S)$        nan    -97.1   139     -55.7          242   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      198   135       102          263   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.469    17    -0.493         16.2   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -3.62    94     -3.14          108   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     8.25    97      4.81          166   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.433  
                $\chi_{c0}(1P)$       -18.6  
                $\chi_{c1}(1P)$         -46  
                $\chi_{c2}(1P)$         112  
                $\eta_{c}(2S)$        -17.9  
$\chi_{c0}(2P)$ $\psi(2S)$            -45.8  
$\chi_{c1}(2P)$ $\psi(2S)$            -97.1  
$\chi_{c2}(2P)$ $\psi(2S)$              198  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.469  
$\chi_{c1}(3P)$ $\psi(2S)$            -3.62  
$\chi_{c2}(3P)$ $\psi(2S)$             8.25

'Approximation: ELW'

,$SR$
$SR-PDG$,98.3
$SR-Deng$,48.9
$SR-\Gamma$,122


16.6

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\chi_{c0}(1P)$ 29.6    -19.2    22     -14.3          30.7   
                $\chi_{c1}(1P)$ 28.3    -32.7    42     -48.6          40.9   
                $\chi_{c2}(1P)$   27     74.6    38       105          41.2   
$\chi_{c0}(2P)$ $\psi(2S)$       nan    -48.2   121     -40.8           143   
$\chi_{c1}(2P)$ $\psi(2S)$       nan     -108   139     -55.7           269   
$\chi_{c2}(2P)$ $\psi(2S)$       nan      228   135       102           303   
$\chi_{c0}(3P)$ $\psi(2S)$       nan   -0.014    17    -0.493         0.484   
$\chi_{c1}(3P)$ $\psi(2S)$       nan     -1.3    94     -3.14          38.8   
$\chi_{c2}(3P)$ $\psi(2S)$       nan     4.44    97      4.81          89.6   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\chi_{c0}(1P)$       -19.9  
                $\chi_{c1}(1P)$       -47.3  
                $\chi_{c2}(1P)$         114  
$\chi_{c0}(2P)$ $\psi(2S)$            -48.2  
$\chi_{c1}(2P)$ $\psi(2S)$             -108  
$\chi_{c2}(2P)$ $\psi(2S)$              228  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.014  
$\chi_{c1}(3P)$ $\psi(2S)$             -1.3  
$\chi_{c2}(3P)$ $\psi(2S)$             4.44

### $\psi(3S)$

In [32]:
reportSumrules("psi_3S", "scr", ("c-scr",))

'Approximation: tot'

,$SR$
$SR-PDG$,-157
$SR-Deng$,-35.3
$SR-\Gamma$,-157


60.975701532214934

PDG  $SR-PDG$  Deng  $SR-Deng$  $\Gamma$  \
In         Out                                                         
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.143  6.76     -0.101      9.62   
           $\chi_{c0}(1P)$  nan     -1.15   6.7     -0.403      19.2   
           $\chi_{c1}(1P)$  nan    -0.399   6.7     -0.273      9.78   
           $\chi_{c2}(1P)$  nan     0.151   2.5      0.195      1.93   
           $\eta_{c}(2S)$   nan    -0.418  1.78     -0.371      2.01   
           $\chi_{c0}(2P)$  nan      -213    27       -179      32.1   
           $\chi_{c1}(2P)$  nan      -132    67       -194      45.7   
           $\chi_{c2}(2P)$  nan       199    82        361      45.2   
           $\eta_{c}(3S)$   nan     -8.78 0.088      -22.5    0.0344   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.143  
           $\chi_{c0}(1P)$        -1.15  
           $\chi_{c1}(1P)$       -0.399  
           $\chi_{c2}(1P)$        0.151  
           $\eta_{c}(2S)$        -0.418  
           $\chi_{c0}(2P)$         -213  
           $\chi_{c1}(2P)$         -132  
           $\chi_{c2}(2P)$          199  
           $\eta_{c}(3S)$         -8.78

'Approximation: E1'

...

PDG  $SR-PDG$  Deng  $SR-Deng$  E1-$\Gamma$  \
In         Out                                                            
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.143  6.76     -0.101         9.62   
           $\chi_{c0}(1P)$  nan    -0.821   6.7     -0.403         13.7   
           $\chi_{c1}(1P)$  nan    -0.399   6.7      -0.32         8.35   
           $\chi_{c2}(1P)$  nan     0.137   2.5      0.153         2.23   
           $\eta_{c}(2S)$   nan    -0.418  1.78     -0.371         2.01   
           $\chi_{c0}(2P)$  nan      -189    27       -179         28.5   
           $\chi_{c1}(2P)$  nan      -132    67       -202         43.9   
           $\chi_{c2}(2P)$  nan       195    82        342         46.7   
           $\eta_{c}(3S)$   nan     -8.78 0.088      -22.5       0.0344   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.143  
           $\chi_{c0}(1P)$       -0.821  
           $\chi_{c1}(1P)$       -0.399  
           $\chi_{c2}(1P)$        0.137  
           $\eta_{c}(2S)$        -0.418  
           $\chi_{c0}(2P)$         -189  
           $\chi_{c1}(2P)$         -132  
           $\chi_{c2}(2P)$          195  
           $\eta_{c}(3S)$         -8.78

'Approximation: ELW'

,$SR$
$SR-PDG$,-147
$SR-Deng$,-40.1
$SR-\Gamma$,-147


68.27787103417218

PDG  $SR-PDG$  Deng  $SR-Deng$  ELW-$\Gamma$  \
In         Out                                                             
$\psi(3S)$ $\chi_{c0}(1P)$  nan    -0.465   6.7     -0.403          7.73   
           $\chi_{c1}(1P)$  nan   -0.0322   6.7      -0.32         0.675   
           $\chi_{c2}(1P)$  nan     0.118   2.5      0.153          1.92   
           $\chi_{c0}(2P)$  nan      -208    27       -179          31.4   
           $\chi_{c1}(2P)$  nan      -138    67       -202          45.7   
           $\chi_{c2}(2P)$  nan       199    82        342          47.9   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\chi_{c0}(1P)$       -0.465  
           $\chi_{c1}(1P)$      -0.0322  
           $\chi_{c2}(1P)$        0.118  
           $\chi_{c0}(2P)$         -208  
           $\chi_{c1}(2P)$         -138  
           $\chi_{c2}(2P)$          199

In [33]:
reportSumrules("psi_3S", "lin", ("c-lin",))

'Approximation: tot'

,$SR$
$SR-PDG$,976
$SR-Deng$,1.17e+03
$SR-\Gamma$,976


57.353873613171075

PDG  $SR-PDG$  Deng  $SR-Deng$  $\Gamma$  \
In         Out                                                         
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.155  6.66    -0.0993      10.4   
           $\chi_{c0}(1P)$  nan      -1.3   5.9     -0.355      21.6   
           $\chi_{c1}(1P)$  nan    -0.407     4     -0.162      10.1   
           $\chi_{c2}(1P)$  nan    0.0608  0.25     0.0196     0.775   
           $\eta_{c}(2S)$   nan    -0.584  1.65     -0.344       2.8   
           $\chi_{c0}(2P)$  nan      -285    19       -126      42.9   
           $\chi_{c1}(2P)$  nan      -345    55       -290      65.3   
           $\chi_{c2}(2P)$  nan       282    67        298      63.4   
           $\eta_{c}(3S)$   nan  1.33e+03 0.051   1.29e+03    0.0527   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.155  
           $\chi_{c0}(1P)$         -1.3  
           $\chi_{c1}(1P)$       -0.407  
           $\chi_{c2}(1P)$       0.0608  
           $\eta_{c}(2S)$        -0.584  
           $\chi_{c0}(2P)$         -285  
           $\chi_{c1}(2P)$         -345  
           $\chi_{c2}(2P)$          282  
           $\eta_{c}(3S)$      1.33e+03

'Approximation: E1'

...

PDG  $SR-PDG$  Deng  $SR-Deng$  E1-$\Gamma$  \
In         Out                                                            
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.155  6.66    -0.0993         10.4   
           $\chi_{c0}(1P)$  nan    -0.922   5.9     -0.355         15.3   
           $\chi_{c1}(1P)$  nan    -0.407     4     -0.191         8.54   
           $\chi_{c2}(1P)$  nan    0.0551  0.25     0.0153          0.9   
           $\eta_{c}(2S)$   nan    -0.584  1.65     -0.344          2.8   
           $\chi_{c0}(2P)$  nan      -251    19       -126         37.8   
           $\chi_{c1}(2P)$  nan      -345    55       -304         62.4   
           $\chi_{c2}(2P)$  nan       274    67        279         65.7   
           $\eta_{c}(3S)$   nan  1.33e+03 0.051   1.29e+03       0.0527   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.155  
           $\chi_{c0}(1P)$       -0.922  
           $\chi_{c1}(1P)$       -0.407  
           $\chi_{c2}(1P)$       0.0551  
           $\eta_{c}(2S)$        -0.584  
           $\chi_{c0}(2P)$         -251  
           $\chi_{c1}(2P)$         -345  
           $\chi_{c2}(2P)$          274  
           $\eta_{c}(3S)$      1.33e+03

'Approximation: ELW'

,$SR$
$SR-PDG$,-358
$SR-Deng$,-152
$SR-\Gamma$,-358


62.343508958094915

PDG  $SR-PDG$  Deng  $SR-Deng$  ELW-$\Gamma$  \
In         Out                                                             
$\psi(3S)$ $\chi_{c0}(1P)$  nan    -0.439   5.9     -0.355           7.3   
           $\chi_{c1}(1P)$  nan    -0.005     4     -0.191         0.105   
           $\chi_{c2}(1P)$  nan     0.434  0.25     0.0153           7.1   
           $\chi_{c0}(2P)$  nan      -279    19       -126          41.9   
           $\chi_{c1}(2P)$  nan      -361    55       -304          65.4   
           $\chi_{c2}(2P)$  nan       282    67        279          67.7   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\chi_{c0}(1P)$       -0.439  
           $\chi_{c1}(1P)$       -0.005  
           $\chi_{c2}(1P)$        0.434  
           $\chi_{c0}(2P)$         -279  
           $\chi_{c1}(2P)$         -361  
           $\chi_{c2}(2P)$          282

In [34]:
reportSumrules("psi_1_1D", "scr", ("c-scr",), sigma=[0]*3)

'Approximation: tot'

,$SR$
$SR-PDG$,-113
$SR-Deng$,-159
$SR-\Gamma$,-156


3.6881474710883997

PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                       
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan  -0.0137   nan   -0.0137     0.427   
                $\chi_{c0}(1P)$  190    -53.2   261     -72.9       275   
                $\chi_{c1}(1P)$ 67.5    -24.5   135     -49.1       119   
                $\chi_{c2}(1P)$  nan     1.84   8.1      2.59      5.74   
                $\eta_{c}(2S)$   nan -0.00097   nan  -0.00097  0.000214   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    -10.7    12     -17.7      7.22   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan      -29   9.8       -23      12.4   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     2.64  0.46      1.63     0.747   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  -0.0551  0.39   -0.0343     0.627   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   -0.304     2    -0.168      3.63   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  -0.0427  0.79   -0.0267      1.26   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$      -0.0137  
                $\chi_{c0}(1P)$       -76.8  
                $\chi_{c1}(1P)$       -43.2  
                $\chi_{c2}(1P)$        1.84  
                $\eta_{c}(2S)$     -0.00097  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -10.7  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$          -29  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         2.64  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$      -0.0551  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$       -0.304  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$      -0.0427

'Approximation: E1'

...

PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan  -0.0137   nan   -0.0137        0.427   
                $\chi_{c0}(1P)$  190    -53.2   261     -72.9          311   
                $\chi_{c1}(1P)$ 67.5    -22.8   135     -45.5          143   
                $\chi_{c2}(1P)$  nan     4.47   8.1      4.67         7.76   
                $\eta_{c}(2S)$   nan -0.00097   nan  -0.00097     0.000214   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    -10.5    12     -17.7         7.08   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    -27.6   9.8     -23.7         11.4   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     1.87  0.46      1.42        0.607   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  -0.0493  0.39   -0.0343         0.56   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   -0.269     2    -0.189         2.84   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   0.0303  0.79      0.11        0.218   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$      -0.0137  
                $\chi_{c0}(1P)$         -87  
                $\chi_{c1}(1P)$       -48.3  
                $\chi_{c2}(1P)$        4.47  
                $\eta_{c}(2S)$     -0.00097  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -10.5  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -27.6  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         1.87  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$      -0.0493  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$       -0.269  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$       0.0303

'Approximation: ELW'

,$SR$
$SR-PDG$,-109
$SR-Deng$,-154
$SR-\Gamma$,-182


8.846176194166047

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\chi_{c0}(1P)$  190    -53.2   261     -72.9           349   
                $\chi_{c1}(1P)$ 67.5    -22.8   135     -45.5           153   
                $\chi_{c2}(1P)$  nan     4.73   8.1      4.67          8.22   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    -10.5    12     -17.7          7.12   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    -28.5   9.8     -23.7          11.8   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     1.96  0.46      1.42         0.638   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.746  0.39   -0.0343          8.48   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan  -0.0335     2    -0.189         0.355   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan     0.01  0.79      0.11        0.0719   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\chi_{c0}(1P)$       -97.4  
                $\chi_{c1}(1P)$       -51.8  
                $\chi_{c2}(1P)$        4.73  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -10.5  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -28.5  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         1.96  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$       -0.746  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$      -0.0335  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$         0.01

In [35]:
reportSumrules("psi_1_1D", "lin", ("c-lin",))

'Approximation: tot'

,$SR$
$SR-PDG$,-125
$SR-Deng$,-164
$SR-\Gamma$,-165


5.6543142112639675

PDG  $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   -0.0117   nan   -0.0117     0.365   
                $\chi_{c0}(1P)$  190     -53.2   272       -76       273   
                $\chi_{c1}(1P)$ 67.5     -24.4   138       -50       115   
                $\chi_{c2}(1P)$  nan      1.83   7.1      2.46      5.28   
                $\eta_{c}(2S)$   nan -0.000628   nan -0.000628  0.000138   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan     -26.1    20     -29.6      17.7   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     -26.8   7.9     -11.5      18.3   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan      3.89  0.36      1.29      1.09   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  -0.00612  0.27   -0.0132     0.126   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan    -0.311   3.2    -0.148      6.71   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   -0.0698   1.5   -0.0387       2.7   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$      -0.0117  
                $\chi_{c0}(1P)$       -76.3  
                $\chi_{c1}(1P)$       -41.6  
                $\chi_{c2}(1P)$        1.83  
                $\eta_{c}(2S)$    -0.000628  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -26.1  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -26.8  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         3.89  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$     -0.00612  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$       -0.311  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$      -0.0698

'Approximation: E1'

...

PDG  $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   -0.0117   nan   -0.0117        0.365   
                $\chi_{c0}(1P)$  190     -53.2   272       -76          307   
                $\chi_{c1}(1P)$ 67.5     -22.8   138     -46.6          137   
                $\chi_{c2}(1P)$  nan      4.05   7.1      4.09         7.03   
                $\eta_{c}(2S)$   nan -0.000628   nan -0.000628     0.000138   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan     -25.4    20     -29.6         17.2   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     -25.3   7.9       -12         16.7   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan      2.61  0.36      1.11        0.849   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  -0.00537  0.27   -0.0132         0.11   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     -0.27   3.2    -0.171         5.07   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan     0.028   1.5     0.116        0.363   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$      -0.0117  
                $\chi_{c0}(1P)$       -85.7  
                $\chi_{c1}(1P)$       -46.3  
                $\chi_{c2}(1P)$        4.05  
                $\eta_{c}(2S)$    -0.000628  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -25.4  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -25.3  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         2.61  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$     -0.00537  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$        -0.27  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$        0.028

'Approximation: ELW'

,$SR$
$SR-PDG$,-121
$SR-Deng$,-159
$SR-\Gamma$,-190


7.767729160715253

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\chi_{c0}(1P)$  190    -53.2   272       -76           339   
                $\chi_{c1}(1P)$ 67.5    -22.8   138     -46.6           146   
                $\chi_{c2}(1P)$  nan     4.26   7.1      4.09          7.39   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    -25.7    20     -29.6          17.4   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    -26.4   7.9       -12          17.4   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     2.79  0.36      1.11         0.908   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.427  0.27   -0.0132          8.75   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan  -0.0175   3.2    -0.171         0.328   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.00675   1.5     0.116        0.0873   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\chi_{c0}(1P)$       -94.8  
                $\chi_{c1}(1P)$       -49.3  
                $\chi_{c2}(1P)$        4.26  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -25.7  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -26.4  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         2.79  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$       -0.427  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$      -0.0175  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$      0.00675

In [36]:
raise KeyboardInterrupt("Enough for output")

KeyboardInterrupt: Enough for output

# Sandbox

In [ ]:
def norm(pname, outdirname):
    cfg = json.load(open(os.path.join("output", outdirname, "config"), "r"), object_pairs_hook=OrderedDict)
    prefix = outdirname.split(".")[0]
    points = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname), index_col=0)
    return sp.sqrt(sp.integrate.trapz(points["u"]**2, points.index.values))

In [ ]:
def plot(ax, pname, outdirname):
    cfg = json.load(open(os.path.join("output", outdirname, "config"), "r"), object_pairs_hook=OrderedDict)
    prefix = outdirname.split(".")[0]
    points = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname), index_col=0)
    norm = sp.sqrt(sp.integrate.trapz(points["u"]**2, points.index.values))
    f = sp.interpolate.interp1d(points.index.values, points["u"])
    ax.plot(points.index.values*0.19732697, points["u"]**2/norm**2/0.19732697, ".")

In [ ]:
def overlap(pname1, pname2, outdirname, func = lambda r: r):
    cfg = json.load(open(os.path.join("output", outdirname, "config"), "r"), object_pairs_hook=OrderedDict)
    prefix = outdirname.split(".")[0]
    points1 = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname1), index_col=0)
    points2 = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname2), index_col=0)
    f1 = sp.interpolate.interp1d(points1.index.values, points1["u"])
    f2 = sp.interpolate.interp1d(points2.index.values, points2["u"])
    if (points1.index.values[-1] > points2.index.values[-1]):
        xes = points2.index.values
    else:
        xes = points1.index.values
    return sp.integrate.trapz(f1(xes)*f2(xes)*func(xes), xes)/sp.sqrt(sp.integrate.trapz(points1["u"]**2, points1.index.values)*sp.integrate.trapz(points2["u"]**2, points2.index.values))

In [ ]:
def readCSV(outdirname):
    data = pd.read_csv(open(os.path.join("output", outdirname, "data")), index_col=[0,1])
    cfg = json.load(open(os.path.join("output", outdirname, "eigen_config"), "r"), object_pairs_hook=OrderedDict)
    def k(row):
        Mf = 2*cfg[row.name[1]]["eq"]["env"]["mC"]+cfg[row.name[1]]["eq"]["E"]
        return -Mf+sp.sqrt(Mf**2 + 2*Mf*(cfg[row.name[0]]["eq"]["E"] - cfg[row.name[1]]["eq"]["E"]))
    lap1 = data.apply(lambda r: overlap(*r.name, outdirname), axis=1).rename("melLW")
    lapful = data.apply(lambda r: overlap(*r.name, outdirname, lambda x: x*(sp.special.spherical_jn(0, k(r)*x/2) - sp.sqrt(5/2)*sp.special.spherical_jn(2, k(r)*x/2))), axis=1).rename("melE1")
    gamma = data.apply(k, axis=1).rename("$E_\gamma$")
    data = pd.concat([data, lap1, lapful, gamma], axis=1)
    check = ((data["widthE1"]/data["melE1"]**2-data["widthELW"]/data["melLW"]**2)*2/(data["widthE1"]/data["melE1"]**2+data["widthELW"]/data["melLW"]**2)).rename("check")
    data = pd.concat([data, check], axis=1)
    return data

In [ ]:
def assembleData(datapath, paperpath):
    chin = pd.read_csv(paperpath, index_col=[0,1]).rename(columns={"Gtot": "widthPAPER"})
    report = readCSV(datapath)
    report = pd.concat([report, chin], axis=1)
    
    return report

In [ ]:
def relativeQuant(report):
    return pd.DataFrame(index=report.index)\
        .join(\
            (report["widthPAPER"]/report["widthE1"])\
                .rename("PAPER/E1"))\
        .join(\
              (report["widthPAPER"]/report["width"])\
                  .rename("PAPER/width"))\
        .join(\
              (report["widthPAPER"]/report["width_mod"])\
                  .rename("PAPER/width_mod"))\
        .join(\
              (report["widthPAPER"]/report["widthE1_mod"])\
                  .rename("PAPER/widthE1_mod"))\
        .join(\
              (report["widthPAPER"]/report["widthELW_mod"])\
                  .rename("PAPER/widthELW_mod")\
             )

In [ ]:
def experimentalReport(report, particledata, transdata):
    trans = pd.read_csv(transdata, index_col=[0, 1]).add_suffix("_pdg")
    pdata = pd.read_csv(particledata, index_col=0)
    width = trans.apply(lambda r: r["frac_width_pdg"]*pdata.loc[r.name[0], "PDG_width"],axis=1).rename("width_pdg")
    eMix = report["$E_\gamma$"].copy()
    collection = []
    collection.append(report[["width", "widthE1", "widthELW", "$E_\gamma$", "widthPAPER"]])
    try:
        notnanE = sp.logical_not(sp.isnan(trans["egamma_pdg"]))
        notnanE = [i for i in notnanE[notnanE].index.values for j in report.index.values if i == j]
        eMix[notnanE] = trans["egamma_pdg"][notnanE].copy()
        collection.append(trans[["egamma_pdg"]])
    except KeyError:
        pass
    eMix.rename("E", inplace=True)
    
    MMix = pdata["PDG"].copy()
    nanM = sp.isnan(MMix)
    nanM = nanM[nanM].index.values
    MMix[nanM] = pdata["SP"][nanM].copy()
    MMix /= 1000
    
    expfactor = eMix.index.map(\
              lambda row: eMix[row]*MMix[row[0]]/(eMix[row] + MMix[row[0]])*(MMix[row[0]] - MMix[row[1]])**2\
        )
    expfactor = pd.DataFrame(expfactor.values, index=eMix.index)
    
    width_mod = (expfactor[0]*report["R_width"].copy()).rename("width_mod")
    widthE1_mod = (expfactor[0]*report["R_widthE1"].copy()).rename("widthE1_mod")
    widthELW_mod = (expfactor[0]*report["R_widthELW"].copy()).rename("widthELW_mod")
    collection += [width_mod, widthE1_mod, widthELW_mod]
    
    return pd.concat(collection, join="inner", axis=1).join(width)

In [ ]:
def doReport(datapath, particledata, paperpath, transdata):
    data = assembleData(datapath, paperpath)
    exper = experimentalReport(data, particledata, transdata)
    rel = (relativeQuant(exper)-1.)*100
    rel.rename(columns={i: i+"(%)" for i in rel.columns}, inplace=True)
    
    labels = ["width", "widthE1", "widthELW", "$E_\gamma$", "widthPAPER"]
    for labe in ["Ge1", "Gm1"]:
        if labe in data.columns:
            labels.append(labe)
    data[labels] *= 10**6
    data.rename(columns=dict(zip(labels, ["%s(KeV)"%i for i in labels])), inplace=True)
    
    labels = ["melLW", "melE1"]
    data[labels] *= 10**(-6)
    data.rename(columns=dict(zip(labels, ["%s($KeV^{-1}$)"%i for i in labels])), inplace=True)
    
    labels = ["R_width", "R_widthE1", "R_widthELW"]
    data[labels] *= 10**(-12)
    data.rename(columns=dict(zip(labels, ["%s($KeV^{-2}$)"%i for i in labels])), inplace=True)
    
    labels = ["width", "widthE1", "widthELW", "width_mod", "widthE1_mod", "widthELW_mod", "$E_\gamma$", "widthPAPER", "width_pdg"]
    for labe in ["egamma_pdg"]:
        if labe in exper.columns:
            labels.append(labe)
    exper[labels] *= 10**6
    exper.rename(columns=dict(zip(labels, ["%s(KeV)"%i for i in labels])), inplace=True)
    
    display(data)
    display(exper)
    display(rel)

In [ ]:
def readTechs(particle, cfgnames, prefix):
    stack = []
    for cfgname in cfgnames:
        stack.append(pd.read_csv("output/{}.{}/melsq".format(prefix, cfgname), header=0, index_col=[0,1], names=["In", "Out", "Tot", "E1", "ELW"]))
    res = pd.concat(stack)
    res = res.query("In == '{0}' | Out == '{0}'".format(particle))
    res.index = pd.Index([tuple("${}$".format(code2name(n)) for n in p) for p in res.index.values], names=["In", "Out"])
    return res

In [ ]:
readTechs("chi_b2_1P", ("1P1D2D.b-scr", "nSnP.b-scr"), "scr")

In [ ]:
doReport("scr.E1.1P1D2D.b-scr", "../data/bottomonia.csv", "../data/b-scr.deng2017-1P1D2D.csv", "../data/bottom-trans-PDG.csv")

In [ ]:
doReport("scr.E1.nSnP.b-scr", "../data/bottomonia.csv", "../data/b-scr.deng2017-nSnP.csv", "../data/bottom-trans-PDG.csv")

In [ ]:
doReport("scr.c-scr","../data/charmonia.csv", "../data/c-scr.deng2017.csv", "../data/charm-trans-PDG.csv")

In [ ]:
doReport("lin.c-lin","../data/charmonia.csv", "../data/c-lin.deng2017.csv", "../data/charm-trans-PDG.csv")